In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

DEVICE CONFIGURATION

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

HYPER-PARAMS

In [3]:
num_epochs = 10
batch_size = 4
learning_rate = 0.001

Dataset has PILImage --> images of range [0,1]

We transform them to Tensors of normalized range [-1, 1]

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [5]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)

Files already downloaded and verified


In [6]:
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)

Files already downloaded and verified


In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size,
                                          shuffle = True)

In [8]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size,
                                          shuffle = False)

In [9]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [10]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
model = ConvNet().to(device)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [13]:
n_total_steps = len(train_loader)

In [14]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # original shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)
        
        #forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 2000 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{n_total_steps}], Loss: {loss.item():.4f}')
            
print('Finished Training')

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value, index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
            
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')
    
    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Epoch [1/10], Step [2000/12500], Loss: 2.2969
Epoch [1/10], Step [4000/12500], Loss: 2.3083
Epoch [1/10], Step [6000/12500], Loss: 2.3357
Epoch [1/10], Step [8000/12500], Loss: 2.1301
Epoch [1/10], Step [10000/12500], Loss: 1.9901
Epoch [1/10], Step [12000/12500], Loss: 2.2307
Epoch [2/10], Step [2000/12500], Loss: 1.8558
Epoch [2/10], Step [4000/12500], Loss: 2.1622
Epoch [2/10], Step [6000/12500], Loss: 2.0053
Epoch [2/10], Step [8000/12500], Loss: 1.7300
Epoch [2/10], Step [10000/12500], Loss: 2.1794
Epoch [2/10], Step [12000/12500], Loss: 2.5101
Epoch [3/10], Step [2000/12500], Loss: 1.7606
Epoch [3/10], Step [4000/12500], Loss: 2.6618
Epoch [3/10], Step [6000/12500], Loss: 2.2233
Epoch [3/10], Step [8000/12500], Loss: 1.8717
Epoch [3/10], Step [10000/12500], Loss: 1.9042
Epoch [3/10], Step [12000/12500], Loss: 1.1378
Epoch [4/10], Step [2000/12500], Loss: 2.9586
Epoch [4/10], Step [4000/12500], Loss: 1.5021
Epoch [4/10], Step [6000/12500], Loss: 1.6368
Epoch [4/10], Step [8000/125